In [ ]:
import numpy as np
import tensorflow as tf
import random
import math

tf.random.set_seed(123)
np.random.seed(123)

In [ ]:
import open3d as o3d
import trimesh
import pyrender
import numpy as np
from PIL import Image

# Загрузка файла PLY
# input_ply = 'Fashion3D/point_cloud/3/3-1.ply'
def ply2obj(class_id, ply_name):
    point_cloud = o3d.io.read_point_cloud(f'Fashion3D/point_cloud/{class_id}/{ply_name}')

    # Оценка нормалей
    point_cloud.estimate_normals()

    # Построение меша с использованием Poisson Surface Reconstruction
    mesh, _ = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(point_cloud, depth=8, n_threads=1)

    # Удаление избыточных частей меша
    bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=(-0.5, -0.5, -0.5), max_bound=(0.5, 0.5, 0.5))
    mesh = mesh.crop(bbox)

    # Центрирование объекта
    mesh.compute_vertex_normals()
    mesh_center = mesh.get_center()
    translation = -mesh_center
    mesh.translate(translation, relative=False)

    # Сохранение результата в файл формата OBJ
    output_obj = f"Fashion3D/obj_dataset/{class_id}/{ply_name.replace('ply', 'obj')}"
    os.makedirs(f"Fashion3D/obj_dataset/{class_id}", exist_ok=True)
    o3d.io.write_triangle_mesh(output_obj, mesh, write_vertex_normals=True)
    
    return output_obj
    
    
    
def get_camera_matrix(rotation_angle, tilt_angle, offset, shift_left=0, shift_right=0, shift_up=0, shift_down=0, roll_angle=0):
    """
    Функция для получения матрицы преобразования координат камеры.
    """
    # Поворачиваем камеру на заданный угол в горизонтальной плоскости.
    rotation_matrix = np.array([
        [np.cos(rotation_angle), 0, np.sin(rotation_angle), 0],
        [0, 1, 0, 0],
        [-np.sin(rotation_angle), 0, np.cos(rotation_angle), 0],
        [0, 0, 0, 1],
    ])

    # Наклоняем камеру на заданный угол в вертикальной плоскости.
    tilt_matrix = np.array([
        [1, 0, 0, 0],
        [0, np.cos(tilt_angle), -np.sin(tilt_angle), 0],
        [0, np.sin(tilt_angle), np.cos(tilt_angle), 0],
        [0, 0, 0, 1],
    ])
    
    # Поворачиваем камеру вокруг оси Z на заданный угол.
    roll_matrix = np.array([
        [np.cos(roll_angle), -np.sin(roll_angle), 0, 0],
        [np.sin(roll_angle), np.cos(roll_angle), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
    ])

    # Смещаем камеру назад на заданное расстояние.
    offset_matrix = np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, offset],
        [0, 0, 0, 1],
    ])

    # Смещаем камеру влево, вправо, вверх и вниз.
    shift_matrix = np.array([
        [1, 0, 0, -shift_left + shift_right],
        [0, 1, 0, -shift_down + shift_up],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
    ])

    # Получаем итоговую матрицу преобразования координат камеры.
    camera_matrix = offset_matrix @ tilt_matrix @ roll_matrix @ rotation_matrix @ shift_matrix

    return camera_matrix


def obj2png_with_aug(obj_file, output_dir, output_ply_dir):
    
    camera_presets = {
        "Поворот на 45 градусов вправо": (np.pi / 4, 0, random.uniform(1.23, 1.27), 0, random.uniform(0.74, 0.76), 0.1, 0., 0),
        "Поворот на 45 градусов влево": (-np.pi / 4, 0, random.uniform(1.23, 1.27), random.uniform(0.79, 0.81), 0, 0.1, 0., 0),
        "Вид спереди под углом": (0, 0, random.uniform(0.97, 1.03), 0, 0, 0.125, 0, random.uniform(-math.pi/4, math.pi/4)),
        "Вид спереди": (0, 0, 1, 0, 0, 0.125, 0., 0),
        "Вид снизу на 45 градусов": (np.pi / 4, np.pi / 4, random.uniform(1.48, 1.52), 0, random.uniform(0.74, 0.76), 0, 0.95, 0),
        "Вид сверху под углом": (np.pi / 8, -np.pi / 4, random.uniform(1.23, 1.27), 0, 0.25, random.uniform(0.98, 1.02), 0, 0)
    }
    
    width, height = 800, 800
    renderer = pyrender.OffscreenRenderer(width, height)  # Создайте экземпляр pyrender.OffscreenRenderer здесь

    for preset_name, _ in camera_presets.items():
        
        rotation_angle, tilt_angle, offset, shift_left, shift_right, shift_up, shift_down, roll_angle = camera_presets[preset_name]
        # Получаем матрицу преобразования координат камеры.
        camera_matrix = get_camera_matrix(rotation_angle, tilt_angle, offset, shift_left, shift_right, shift_up, shift_down, roll_angle)

        mesh = trimesh.load(obj_file)

        # Создайте сцену для рендеринга
        scene = pyrender.Scene()

        # Добавьте меш с вершинными цветами в сцену
        material = pyrender.MetallicRoughnessMaterial(
            metallicFactor=0.0,
            alphaMode='OPAQUE',
            baseColorFactor=(1.0, 1.0, 1.0, 1.0),
        )
        scene.add(pyrender.Mesh.from_trimesh(mesh, material=material))

        # Установите камеру
        camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0)
        camera_pose = np.array(camera_matrix)
        scene.add(camera, pose=camera_pose)

        # Добавьте источник света
        light = pyrender.PointLight(color=np.ones(3), intensity=50.0)
        light_pose = np.array([
            [1, 0, 0, 0],
            [0, 1, 0, 3],
            [0, 0, 1, 3],
            [0, 0, 0, 1],
        ])
        scene.add(light, pose=light_pose)

        # Рендеринг сцены
        color, depth = renderer.render(scene)

        # Сохраните изображение в формате PNG
        image = Image.fromarray(color)
        image.save(os.path.join(output_dir, f"{preset_name}.png"))
    
    renderer.delete()

In [ ]:
def parse_class_ids(file_name):
    class_ids = {}
    with open(file_name, 'r') as file:
        lines = file.readlines()
        for line in lines:
            line_data = line.strip().split()
            class_name = line_data[0]
            ids = [int(i) for i in line_data[1:]]
            class_ids[class_name] = ids
    return class_ids

def scatter_to_image(x_view, y_view, colors, point_size):
    fig, ax = plt.subplots()
    ax.scatter(x_view, y_view, c=colors, s=point_size)
    ax.axis("equal")
    ax.set_aspect('equal', adjustable='box')
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    fig.canvas.draw()
    img_arr = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    img_arr = img_arr.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.close(fig)
    return img_arr

In [ ]:
import os
import numpy as np
import cv2
import trimesh
import open3d as o3d
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO

file_name = 'Fashion3D/cloth_type_list.txt'
class_idss = parse_class_ids(file_name)

root_dir = "Fashion3D"

for class_name, class_ids in class_idss.items():
    for class_id in class_ids:
        input_dir = os.path.join(root_dir, "obj_dataset", str(class_id))
        output_dir = os.path.join(root_dir, "2D_representation_new", str(class_id))
        if os.path.exists(input_dir) and not os.path.exists(output_dir):
            for file_name in os.listdir(input_dir):
                if file_name.endswith(".obj") and 'aug' not in file_name:
                    obj_file = f"Fashion3D/obj_dataset/{class_id}/{file_name.replace('ply', 'obj')}"
                    output_subdir = os.path.join(output_dir, file_name[:-4])
                    os.makedirs(output_subdir, exist_ok=True)
                    print(obj_file, output_subdir, input_dir)

                    obj2png_with_aug(obj_file, output_subdir, input_dir)

In [ ]:
import os
import glob

representation_root = "Fashion3D/2D_representation_new"
point_cloud_root = "Fashion3D/point_cloud"


data = []
for class_id in os.listdir(representation_root):
    class_dir = os.path.join(point_cloud_root, str(class_id))
    ply_files = glob.glob(os.path.join(class_dir, "*.ply"))
    
    for ply_file in ply_files:
        
        ply_file_name = os.path.basename(ply_file)

        representation_dir = os.path.join(representation_root, str(class_id), ply_file_name[:-4])
        jpg_files = glob.glob(os.path.join(representation_dir, "*.png"))

        for jpg_file in jpg_files:
            if ('Вид спереди' in jpg_file and 'Вид спереди под углом' not in jpg_file):
                jpg_file_name = os.path.basename(jpg_file)

                data.append((jpg_file, ply_file))

In [ ]:
def load_png_file(file_path):
    """
    Загрузка PNG-файла и препроцессинг

    Аргументы:
    file_path (str): Путь к PNG-файлу

    Возвращает:
    square_data (np.array): Массив с данными из PNG-файла
    """
    # Загружаем PNG-файл с использованием OpenCV
    png_data = cv2.imread(file_path)

    # Преобразуем изображение в оттенки серого
    grayscale_data = cv2.cvtColor(png_data, cv2.COLOR_RGB2GRAY)
    
    # Используем пороговое значение (thresholding) для создания бинарного изображения
    _, binary_data = cv2.threshold(grayscale_data, 250, 255, cv2.THRESH_BINARY_INV)

    # Находим контуры объектов на бинарном изображении
    contours, _ = cv2.findContours(binary_data, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Вычисляем границы области интереса
    x_min, y_min, x_max, y_max = np.inf, np.inf, 0, 0
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        x_min = min(x_min, x)
        y_min = min(y_min, y)
        x_max = max(x_max, x + w)
        y_max = max(y_max, y + h)

    # Вычисляем размеры области интереса и определяем размер квадрата
    width, height = x_max - x_min, y_max - y_min
    square_size = max(width, height)

    # Вычисляем координаты верхнего левого угла квадрата
    x_start = max(0, x_min + (width - square_size) // 2)
    y_start = max(0, y_min + (height - square_size) // 2)

    # Вырезаем квадратное изображение
    square_data = grayscale_data[y_start:y_start + square_size, x_start:x_start + square_size]

    # Изменяем размер квадратного изображения на 127x127
    square_data = cv2.resize(square_data, (127, 127))

    # Нормализуем значения пикселей
    square_data = square_data.astype(np.float32) / 255.0
    square_data = np.expand_dims(square_data, axis=2)

    return square_data








import open3d as o3d
import numpy as np


def load_ply_file(file_path, resolution=32):
    pcd = o3d.io.read_point_cloud(file_path)
    xyz_load = np.asarray(pcd.points)

    # Вычисление минимальных и максимальных координат
    min_coords = np.min(xyz_load, axis=0)
    max_coords = np.max(xyz_load, axis=0)

    # Вычислить текущий диапазон значений по каждой оси
    ranges = max_coords - min_coords

    # Найти максимальный диапазон среди всех осей
    max_range = np.max(ranges)

    # Вычислить масштабный коэффициент для каждой оси
    scale_factor = (resolution - 1) / max_range

    # Вычитать минимальные значения каждой оси из всех точек и умножить их на масштабный коэффициент
    scaled_points = (xyz_load - min_coords) * scale_factor
    scaled_points = np.round(scaled_points).astype(int)

    # Создание пустого массива размерности (32, 32, 32)
    voxel_array = np.zeros((resolution, resolution, resolution, 1), dtype=np.uint8)

    # Заполнение массива на основе координат точек
    for point in scaled_points:
        x, y, z = point
        voxel_array[x, y, z] = 1

    return voxel_array





def preprocess_data(data):
    x = []
    y = []
        
    for jpeg_path, ply_path in data:
        jpeg_data = load_png_file(jpeg_path)
        ply_data = load_ply_file(ply_path)
        x.append(jpeg_data)
        y.append(ply_data)
    return np.asarray(x), np.asarray(y)

def prepare_data(data, test_size=0.1, random_state=123):
    x, y = preprocess_data(data)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=random_state)
    return x_train, x_test, y_train, y_test 

In [ ]:
x_train, x_test, y_train, y_test = prepare_data(data)

In [ ]:
import tensorflow as tf
from tensorflow.keras import Model, layers

class Conv2DWithBatchNormalizationWithRelu(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size, strides=(1, 1), use_batch_norm=False, name=None, kernel_initializer='glorot_uniform', kernel_regularizer=None, **kwargs):
        super(Conv2DWithBatchNormalizationWithRelu, self).__init__(name=name, **kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.strides = strides
        self.use_batch_norm = use_batch_norm
        self.kernel_initializer = kernel_initializer
        self.kernel_regularizer = kernel_regularizer

    def build(self, input_shape):
        self.conv2d_weights = self.add_weight(name=f"{self.name}_conv2d_weights",
                                              shape=(self.kernel_size[0], self.kernel_size[1], input_shape[-1], self.filters),
                                              initializer=self.kernel_initializer,
                                              regularizer=self.kernel_regularizer,
                                              trainable=True)
        if self.use_batch_norm:
            self.batch_mean = tf.Variable(tf.zeros(self.filters), name=f"{self.name}_batch_mean")
            self.batch_var = tf.Variable(tf.ones(self.filters), name=f"{self.name}_batch_var")
            self.beta = tf.Variable(tf.zeros(self.filters), name=f"{self.name}_beta")
            self.gamma = tf.Variable(tf.ones(self.filters), name=f"{self.name}_gamma")

    def call(self, inputs, training=None):
        x = tf.nn.conv2d(inputs, self.conv2d_weights, strides=[1, *self.strides, 1], padding='SAME', name=f"{self.name}_conv2d")
        
        if self.use_batch_norm:
            mean, variance = tf.nn.moments(x, axes=[0, 1, 2], keepdims=False)

            if training:
                x_normalized = (x - mean) / tf.math.sqrt(variance + 1e-5)
            else:
                x_normalized = (x - self.batch_mean) / tf.math.sqrt(self.batch_var + 1e-5)

            x = self.gamma * x_normalized + self.beta
        
        y = tf.nn.leaky_relu(x, alpha=0.1, name=f"{self.name}_leaky_relu")
        return y

    def get_config(self):
        config = super().get_config()
        config.update({
            "filters": self.filters,
            "kernel_size": self.kernel_size,
            "strides": self.strides,
            "use_batch_norm": self.use_batch_norm,
            "kernel_initializer": self.kernel_initializer,
            "kernel_regularizer": self.kernel_regularizer
        })
        return config

In [ ]:
from tensorflow.keras.regularizers import L1L2
from keras import initializers

l1_coeff = 0.000
l2_coeff = 0.000

batch_size = 32
initializer = initializers.HeNormal(seed=123)
regularizer = tf.keras.regularizers.l1_l2(l1=l1_coeff, l2=l2_coeff)


inputs = tf.keras.Input(shape=[127, 127, 1], batch_size=batch_size, name='input_layer')

###
conv0_0 = Conv2DWithBatchNormalizationWithRelu(filters=64, kernel_size=(7, 7), strides=(1, 1), kernel_initializer=initializer, kernel_regularizer=regularizer, name='conv0_0')(inputs)
conv0_1 = Conv2DWithBatchNormalizationWithRelu(filters=64, kernel_size=(3, 3), strides=(1, 1), kernel_initializer=initializer, kernel_regularizer=regularizer, name='conv0_1')(conv0_0)
shortcut0 = Conv2DWithBatchNormalizationWithRelu(filters=64, kernel_size=(1, 1), strides=(1, 1), kernel_initializer=initializer, kernel_regularizer=regularizer, name='shortcut0')(inputs)
output0 = tf.nn.max_pool(conv0_1 + shortcut0, ksize=(2, 2), strides=(2, 2), padding='SAME', name='output0')

conv1_0 = Conv2DWithBatchNormalizationWithRelu(filters=64, kernel_size=(3, 3), strides=(1, 1), kernel_initializer=initializer, kernel_regularizer=regularizer, name='conv1_0')(output0)
conv1_1 = Conv2DWithBatchNormalizationWithRelu(filters=64, kernel_size=(3, 3), strides=(1, 1), kernel_initializer=initializer, kernel_regularizer=regularizer, name='conv1_1')(conv1_0)
shortcut1 = Conv2DWithBatchNormalizationWithRelu(filters=64, kernel_size=(1, 1), strides=(1, 1), kernel_initializer=initializer, kernel_regularizer=regularizer, name='shortcut1')(output0)
output1 = tf.nn.max_pool(conv1_1 + shortcut1, ksize=(2, 2), strides=(2, 2), padding='SAME', name='output1')

conv2_0 = Conv2DWithBatchNormalizationWithRelu(filters=128, kernel_size=(3, 3), strides=(1, 1), kernel_initializer=initializer, kernel_regularizer=regularizer, name='conv2_0')(output1)
conv2_1 = Conv2DWithBatchNormalizationWithRelu(filters=128, kernel_size=(3, 3), strides=(1, 1), kernel_initializer=initializer, kernel_regularizer=regularizer, name='conv2_1')(conv2_0)
shortcut2 = Conv2DWithBatchNormalizationWithRelu(filters=128, kernel_size=(1, 1), strides=(1, 1), kernel_initializer=initializer, kernel_regularizer=regularizer, name='shortcut2')(output1)
output2 = tf.nn.max_pool(conv2_1 + shortcut2, ksize=(2, 2), strides=(2, 2), padding='SAME', name='output2')

conv3_0 = Conv2DWithBatchNormalizationWithRelu(filters=128, kernel_size=(3, 3), strides=(1, 1), kernel_initializer=initializer, kernel_regularizer=regularizer, name='conv3_0')(output2)
conv3_1 = Conv2DWithBatchNormalizationWithRelu(filters=128, kernel_size=(3, 3), strides=(1, 1), kernel_initializer=initializer, kernel_regularizer=regularizer, name='conv3_1')(conv3_0)
output3 = tf.nn.max_pool(conv3_1, ksize=(2, 2), strides=(2, 2), padding='SAME', name='output3')

conv4_0 = Conv2DWithBatchNormalizationWithRelu(filters=256, kernel_size=(3, 3), strides=(1, 1), kernel_initializer=initializer, kernel_regularizer=regularizer, name='conv4_0')(output3)
conv4_1 = Conv2DWithBatchNormalizationWithRelu(filters=256, kernel_size=(3, 3), strides=(1, 1), kernel_initializer=initializer, kernel_regularizer=regularizer, name='conv4_1')(conv4_0)
shortcut4 = Conv2DWithBatchNormalizationWithRelu(filters=256, kernel_size=(1, 1), strides=(1, 1), kernel_initializer=initializer, kernel_regularizer=regularizer, name='shortcut4')(output3)
output4 = tf.nn.max_pool(conv4_1 + shortcut4, ksize=(2, 2), strides=(2, 2), padding='SAME', name='output4')

conv5_0 = Conv2DWithBatchNormalizationWithRelu(filters=256, kernel_size=(3, 3), strides=(1, 1), kernel_initializer=initializer, kernel_regularizer=regularizer, name='conv5_0')(output4)
conv5_1 = Conv2DWithBatchNormalizationWithRelu(filters=256, kernel_size=(3, 3), strides=(1, 1), kernel_initializer=initializer, kernel_regularizer=regularizer, name='conv5_1')(conv5_0)
shortcut5 = Conv2DWithBatchNormalizationWithRelu(filters=256, kernel_size=(1, 1), strides=(1, 1), kernel_initializer=initializer, kernel_regularizer=regularizer, name='shortcut5')(output4)
output5 = tf.nn.max_pool(conv5_1 + shortcut5, ksize=(2, 2), strides=(1, 1), padding='SAME', name='output5')

decoder_input = layers.Reshape((2, 2, 2, 512), name='reshape_decoder_input')(output5)

unpool0 = tf.keras.layers.UpSampling3D(size=(2, 2, 2), name='unpool0')(decoder_input)
deshortcut0 = tf.keras.layers.Conv3D(filters=256, kernel_size=1, strides=(1, 1, 1), padding='same', kernel_initializer=initializer, kernel_regularizer=L1L2(l1=l1_coeff, l2=l2_coeff), activation='relu', name='deshortcut0')(unpool0)
deconv0_0 = tf.keras.layers.Conv3D(filters=256, kernel_size=3, strides=(1, 1, 1), padding='same', kernel_initializer=initializer, kernel_regularizer=L1L2(l1=l1_coeff, l2=l2_coeff), activation='relu', name='deconv0_0')(deshortcut0)
deconv0_1 = tf.keras.layers.Conv3D(filters=256, kernel_size=3, strides=(1, 1, 1), padding='same', kernel_initializer=initializer, kernel_regularizer=L1L2(l1=l1_coeff, l2=l2_coeff), activation='relu', name='deconv0_1')(deconv0_0)
output0 = tf.add(deshortcut0, deconv0_1, name='dec_output0')


unpool1 = tf.keras.layers.UpSampling3D(size=(2, 2, 2), name='unpool1')(output0)
deshortcut1 = tf.keras.layers.Conv3D(filters=128, kernel_size=1, strides=(1, 1, 1), padding='same', kernel_initializer=initializer, kernel_regularizer=L1L2(l1=l1_coeff, l2=l2_coeff), activation='relu', name='deshortcut1')(unpool1)
deconv1_0 = tf.keras.layers.Conv3D(filters=128, kernel_size=3, strides=(1, 1, 1), padding='same', kernel_initializer=initializer, kernel_regularizer=L1L2(l1=l1_coeff, l2=l2_coeff), activation='relu', name='deconv1_0')(deshortcut1)
deconv1_1 = tf.keras.layers.Conv3D(filters=128, kernel_size=3, strides=(1, 1, 1), padding='same', kernel_initializer=initializer, kernel_regularizer=L1L2(l1=l1_coeff, l2=l2_coeff), activation='relu', name='deconv1_1')(deconv1_0)
output1 = tf.add(deshortcut1, deconv1_1, name='dec_output1')

unpool2 = tf.keras.layers.UpSampling3D(size=(2, 2, 2), name='unpool2')(output1)
deshortcut2 = tf.keras.layers.Conv3D(filters=64, kernel_size=1, strides=(1, 1, 1), padding='same', kernel_initializer=initializer, kernel_regularizer=L1L2(l1=l1_coeff, l2=l2_coeff), activation='relu', name='deshortcut2')(unpool2)
deconv2_0 = tf.keras.layers.Conv3D(filters=64, kernel_size=3, strides=(1, 1, 1), padding='same', kernel_initializer=initializer, kernel_regularizer=L1L2(l1=l1_coeff, l2=l2_coeff), activation='relu', name='deconv2_0')(unpool2)
deconv2_1 = tf.keras.layers.Conv3D(filters=64, kernel_size=3, strides=(1, 1, 1), padding='same', kernel_initializer=initializer, kernel_regularizer=L1L2(l1=l1_coeff, l2=l2_coeff), activation='relu', name='deconv2_1')(deconv2_0)
output2 = tf.add(deshortcut2, deconv2_1, name='dec_output2')

unpool3 = tf.keras.layers.UpSampling3D(size=(2, 2, 2), name='unpool3')(output2)
deshortcut3 = tf.keras.layers.Conv3D(filters=32, kernel_size=1, strides=(1, 1, 1), padding='same', kernel_initializer=initializer, kernel_regularizer=L1L2(l1=l1_coeff, l2=l2_coeff), activation='relu', name='deshortcut3')(unpool3)
deconv3_0 = tf.keras.layers.Conv3D(filters=32, kernel_size=3, strides=(1, 1, 1), padding='same', kernel_initializer=initializer, kernel_regularizer=L1L2(l1=l1_coeff, l2=l2_coeff), activation='relu', name='deconv3_0')(deshortcut3)
deconv3_1 = tf.keras.layers.Conv3D(filters=32, kernel_size=3, strides=(1, 1, 1), padding='same', kernel_initializer=initializer, kernel_regularizer=L1L2(l1=l1_coeff, l2=l2_coeff), activation='relu', name='deconv3_1')(deconv3_0)
deconv3_2 = tf.keras.layers.Conv3D(filters=32, kernel_size=3, strides=(1, 1, 1), padding='same', kernel_initializer=initializer, kernel_regularizer=L1L2(l1=l1_coeff, l2=l2_coeff), activation='relu', name='deconv3_2')(deconv3_1)
output3 = tf.add(deshortcut3, deconv3_2, name='dec_output3')

deshortcut4 = tf.keras.layers.Conv3D(filters=16, kernel_size=1, strides=(1, 1, 1), padding='same', kernel_initializer=initializer, kernel_regularizer=L1L2(l1=l1_coeff, l2=l2_coeff), activation='relu', name='deshortcut4')(output3)
deconv4_0 = tf.keras.layers.Conv3D(filters=16, kernel_size=3, strides=(1, 1, 1), padding='same', kernel_initializer=initializer, kernel_regularizer=L1L2(l1=l1_coeff, l2=l2_coeff), activation='relu', name='deconv4_0')(deshortcut4)
deconv4_1 = tf.keras.layers.Conv3D(filters=16, kernel_size=3, strides=(1, 1, 1), padding='same', kernel_initializer=initializer, kernel_regularizer=L1L2(l1=l1_coeff, l2=l2_coeff), activation='relu', name='deconv4_1')(deconv4_0)
output4 = tf.add(deshortcut4, deconv4_1, name='dec_output4')

output = tf.keras.layers.Conv3D(filters=1, kernel_size=3, strides=(1, 1, 1), padding='same', kernel_initializer=initializer, kernel_regularizer=L1L2(l1=l1_coeff, l2=l2_coeff), activation='sigmoid', name='dec_output4')(output4)

In [ ]:
import keras.backend as K
import tensorflow as tf
import tensorflow.keras.backend as K

loss = tf.keras.losses.BinaryCrossentropy()


def batch_iou(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(K.clip(y_pred, 0, 1), 'float32')
    
    y_true_f = K.batch_flatten(y_true)
    y_pred_f = K.batch_flatten(y_pred)
    
    intersection = K.sum(y_true_f * y_pred_f, axis=-1)
    union = K.sum(y_true_f, axis=-1) + K.sum(y_pred_f, axis=-1) - intersection
    
    jac = (intersection + 0.00001) / (union + 0.00001)
    
    return K.mean(jac)

In [ ]:
from keras.losses import Loss
import os
import logging
from tensorflow.keras.metrics import BinaryAccuracy

optimizer = tf.keras.optimizers.legacy.Adam(
    learning_rate=0.0005704251931830293,
    beta_1=0.969977395782997,
    beta_2=0.9924554943638575,
    epsilon=2.463546078002056e-09
)


        
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'  # 0 = default (all messages), 1 = info and warnings, 2 = warnings and errors, 3 = errors only
tf.get_logger().setLevel(logging.ERROR)

In [ ]:
from keras_tuner import HyperModel
from tensorflow.keras.regularizers import L1L2
from keras import initializers
from tensorflow.keras.metrics import BinaryAccuracy

# Класс для подбора гиперпараметров по сетке

class CustomHyperModel(HyperModel):
    def __init__(self):
        self.all_histories = [] 
    
    def fit(self, hp, model, *args, **kwargs):
        filename = "training_metrics_FINAL.txt"
        mode = "w" if not os.path.exists(filename) else "a"
        
        history = model.fit(*args, **kwargs)
        self.all_histories.append(history)  
        val_loss = history.history['val_loss'][-1]

        with open(filename, mode) as f:
            f.write(f"Epoch {len(self.all_histories)}:\n")
            for key, value in history.history.items():
                f.write(f"{key}: {value[-1]}\n")
            f.write("\n")
        
        return val_loss

    def build(self, hp):
    
        loss = tf.keras.losses.BinaryCrossentropy()
                        
        l1_coeff = 0
        l2_coeff = 0

        
        initializer_choice = hp.Choice('initializer', ['he_normal', 'lecun_normal', 'glorot_normal'])

        if initializer_choice == 'he_normal':
            initializer = initializers.HeNormal(seed=123)
        elif initializer_choice == 'lecun_normal':
            initializer = initializers.LecunNormal(seed=123)
        else:
            initializer = initializers.GlorotNormal(seed=123)
            

        batch_size = 32
        regularizer = tf.keras.regularizers.l1_l2(l1=l1_coeff, l2=l2_coeff)


        inputs = tf.keras.Input(shape=[127, 127, 1], batch_size=batch_size, name='input_layer')

        ###
        conv0_0 = Conv2DWithBatchNormalizationWithRelu(filters=64, kernel_size=(7, 7), strides=(1, 1), kernel_initializer=initializer, kernel_regularizer=regularizer, name='conv0_0')(inputs)
        conv0_1 = Conv2DWithBatchNormalizationWithRelu(filters=64, kernel_size=(3, 3), strides=(1, 1), kernel_initializer=initializer, kernel_regularizer=regularizer, name='conv0_1')(conv0_0)
        shortcut0 = Conv2DWithBatchNormalizationWithRelu(filters=64, kernel_size=(1, 1), strides=(1, 1), kernel_initializer=initializer, kernel_regularizer=regularizer, name='shortcut0')(inputs)
        output0 = tf.nn.max_pool(conv0_1 + shortcut0, ksize=(2, 2), strides=(2, 2), padding='SAME', name='output0')

        conv1_0 = Conv2DWithBatchNormalizationWithRelu(filters=64, kernel_size=(3, 3), strides=(1, 1), kernel_initializer=initializer, kernel_regularizer=regularizer, name='conv1_0')(output0)
        conv1_1 = Conv2DWithBatchNormalizationWithRelu(filters=64, kernel_size=(3, 3), strides=(1, 1), kernel_initializer=initializer, kernel_regularizer=regularizer, name='conv1_1')(conv1_0)
        shortcut1 = Conv2DWithBatchNormalizationWithRelu(filters=64, kernel_size=(1, 1), strides=(1, 1), kernel_initializer=initializer, kernel_regularizer=regularizer, name='shortcut1')(output0)
        output1 = tf.nn.max_pool(conv1_1 + shortcut1, ksize=(2, 2), strides=(2, 2), padding='SAME', name='output1')

        conv2_0 = Conv2DWithBatchNormalizationWithRelu(filters=128, kernel_size=(3, 3), strides=(1, 1), kernel_initializer=initializer, kernel_regularizer=regularizer, name='conv2_0')(output1)
        conv2_1 = Conv2DWithBatchNormalizationWithRelu(filters=128, kernel_size=(3, 3), strides=(1, 1), kernel_initializer=initializer, kernel_regularizer=regularizer, name='conv2_1')(conv2_0)
        shortcut2 = Conv2DWithBatchNormalizationWithRelu(filters=128, kernel_size=(1, 1), strides=(1, 1), kernel_initializer=initializer, kernel_regularizer=regularizer, name='shortcut2')(output1)
        output2 = tf.nn.max_pool(conv2_1 + shortcut2, ksize=(2, 2), strides=(2, 2), padding='SAME', name='output2')

        conv3_0 = Conv2DWithBatchNormalizationWithRelu(filters=128, kernel_size=(3, 3), strides=(1, 1), kernel_initializer=initializer, kernel_regularizer=regularizer, name='conv3_0')(output2)
        conv3_1 = Conv2DWithBatchNormalizationWithRelu(filters=128, kernel_size=(3, 3), strides=(1, 1), kernel_initializer=initializer, kernel_regularizer=regularizer, name='conv3_1')(conv3_0)
        output3 = tf.nn.max_pool(conv3_1, ksize=(2, 2), strides=(2, 2), padding='SAME', name='output3')

        conv4_0 = Conv2DWithBatchNormalizationWithRelu(filters=256, kernel_size=(3, 3), strides=(1, 1), kernel_initializer=initializer, kernel_regularizer=regularizer, name='conv4_0')(output3)
        conv4_1 = Conv2DWithBatchNormalizationWithRelu(filters=256, kernel_size=(3, 3), strides=(1, 1), kernel_initializer=initializer, kernel_regularizer=regularizer, name='conv4_1')(conv4_0)
        shortcut4 = Conv2DWithBatchNormalizationWithRelu(filters=256, kernel_size=(1, 1), strides=(1, 1), kernel_initializer=initializer, kernel_regularizer=regularizer, name='shortcut4')(output3)
        output4 = tf.nn.max_pool(conv4_1 + shortcut4, ksize=(2, 2), strides=(2, 2), padding='SAME', name='output4')

        conv5_0 = Conv2DWithBatchNormalizationWithRelu(filters=256, kernel_size=(3, 3), strides=(1, 1), kernel_initializer=initializer, kernel_regularizer=regularizer, name='conv5_0')(output4)
        conv5_1 = Conv2DWithBatchNormalizationWithRelu(filters=256, kernel_size=(3, 3), strides=(1, 1), kernel_initializer=initializer, kernel_regularizer=regularizer, name='conv5_1')(conv5_0)
        shortcut5 = Conv2DWithBatchNormalizationWithRelu(filters=256, kernel_size=(1, 1), strides=(1, 1), kernel_initializer=initializer, kernel_regularizer=regularizer, name='shortcut5')(output4)
        output5 = tf.nn.max_pool(conv5_1 + shortcut5, ksize=(2, 2), strides=(1, 1), padding='SAME', name='output5')
        
        decoder_input = layers.Reshape((2, 2, 2, 512), name='reshape_decoder_input')(output5)

        unpool0 = tf.keras.layers.UpSampling3D(size=(2, 2, 2), name='unpool0')(decoder_input)
        deshortcut0 = tf.keras.layers.Conv3D(filters=256, kernel_size=1, strides=(1, 1, 1), padding='same', kernel_initializer=initializer, kernel_regularizer=L1L2(l1=l1_coeff, l2=l2_coeff), activation='relu', name='deshortcut0')(unpool0)
        deconv0_0 = tf.keras.layers.Conv3D(filters=256, kernel_size=3, strides=(1, 1, 1), padding='same', kernel_initializer=initializer, kernel_regularizer=L1L2(l1=l1_coeff, l2=l2_coeff), activation='relu', name='deconv0_0')(deshortcut0)
        deconv0_1 = tf.keras.layers.Conv3D(filters=256, kernel_size=3, strides=(1, 1, 1), padding='same', kernel_initializer=initializer, kernel_regularizer=L1L2(l1=l1_coeff, l2=l2_coeff), activation='relu', name='deconv0_1')(deconv0_0)
        output0 = tf.add(deshortcut0, deconv0_1, name='dec_output0')


        unpool1 = tf.keras.layers.UpSampling3D(size=(2, 2, 2), name='unpool1')(output0)
        deshortcut1 = tf.keras.layers.Conv3D(filters=128, kernel_size=1, strides=(1, 1, 1), padding='same', kernel_initializer=initializer, kernel_regularizer=L1L2(l1=l1_coeff, l2=l2_coeff), activation='relu', name='deshortcut1')(unpool1)
        deconv1_0 = tf.keras.layers.Conv3D(filters=128, kernel_size=3, strides=(1, 1, 1), padding='same', kernel_initializer=initializer, kernel_regularizer=L1L2(l1=l1_coeff, l2=l2_coeff), activation='relu', name='deconv1_0')(deshortcut1)
        deconv1_1 = tf.keras.layers.Conv3D(filters=128, kernel_size=3, strides=(1, 1, 1), padding='same', kernel_initializer=initializer, kernel_regularizer=L1L2(l1=l1_coeff, l2=l2_coeff), activation='relu', name='deconv1_1')(deconv1_0)
        output1 = tf.add(deshortcut1, deconv1_1, name='dec_output1')

        unpool2 = tf.keras.layers.UpSampling3D(size=(2, 2, 2), name='unpool2')(output1)
        deshortcut2 = tf.keras.layers.Conv3D(filters=64, kernel_size=1, strides=(1, 1, 1), padding='same', kernel_initializer=initializer, kernel_regularizer=L1L2(l1=l1_coeff, l2=l2_coeff), activation='relu', name='deshortcut2')(unpool2)
        deconv2_0 = tf.keras.layers.Conv3D(filters=64, kernel_size=3, strides=(1, 1, 1), padding='same', kernel_initializer=initializer, kernel_regularizer=L1L2(l1=l1_coeff, l2=l2_coeff), activation='relu', name='deconv2_0')(unpool2)
        deconv2_1 = tf.keras.layers.Conv3D(filters=64, kernel_size=3, strides=(1, 1, 1), padding='same', kernel_initializer=initializer, kernel_regularizer=L1L2(l1=l1_coeff, l2=l2_coeff), activation='relu', name='deconv2_1')(deconv2_0)
        output2 = tf.add(deshortcut2, deconv2_1, name='dec_output2')

        unpool3 = tf.keras.layers.UpSampling3D(size=(2, 2, 2), name='unpool3')(output2)
        deshortcut3 = tf.keras.layers.Conv3D(filters=32, kernel_size=1, strides=(1, 1, 1), padding='same', kernel_initializer=initializer, kernel_regularizer=L1L2(l1=l1_coeff, l2=l2_coeff), activation='relu', name='deshortcut3')(unpool3)
        deconv3_0 = tf.keras.layers.Conv3D(filters=32, kernel_size=3, strides=(1, 1, 1), padding='same', kernel_initializer=initializer, kernel_regularizer=L1L2(l1=l1_coeff, l2=l2_coeff), activation='relu', name='deconv3_0')(deshortcut3)
        deconv3_1 = tf.keras.layers.Conv3D(filters=32, kernel_size=3, strides=(1, 1, 1), padding='same', kernel_initializer=initializer, kernel_regularizer=L1L2(l1=l1_coeff, l2=l2_coeff), activation='relu', name='deconv3_1')(deconv3_0)
        deconv3_2 = tf.keras.layers.Conv3D(filters=32, kernel_size=3, strides=(1, 1, 1), padding='same', kernel_initializer=initializer, kernel_regularizer=L1L2(l1=l1_coeff, l2=l2_coeff), activation='relu', name='deconv3_2')(deconv3_1)
        output3 = tf.add(deshortcut3, deconv3_2, name='dec_output3')

        deshortcut4 = tf.keras.layers.Conv3D(filters=16, kernel_size=1, strides=(1, 1, 1), padding='same', kernel_initializer=initializer, kernel_regularizer=L1L2(l1=l1_coeff, l2=l2_coeff), activation='relu', name='deshortcut4')(output3)
        deconv4_0 = tf.keras.layers.Conv3D(filters=16, kernel_size=3, strides=(1, 1, 1), padding='same', kernel_initializer=initializer, kernel_regularizer=L1L2(l1=l1_coeff, l2=l2_coeff), activation='relu', name='deconv4_0')(deshortcut4)
        deconv4_1 = tf.keras.layers.Conv3D(filters=16, kernel_size=3, strides=(1, 1, 1), padding='same', kernel_initializer=initializer, kernel_regularizer=L1L2(l1=l1_coeff, l2=l2_coeff), activation='relu', name='deconv4_1')(deconv4_0)
        output4 = tf.add(deshortcut4, deconv4_1, name='dec_output4')

        output = tf.keras.layers.Conv3D(filters=1, kernel_size=3, strides=(1, 1, 1), padding='same', kernel_initializer=initializer, kernel_regularizer=L1L2(l1=l1_coeff, l2=l2_coeff), activation='sigmoid', name='dec_output4')(output4)

        
        model = tf.keras.Model(inputs=inputs, outputs=output)
        loss = tf.keras.losses.BinaryCrossentropy()
        model.compile(
            optimizer=tf.keras.optimizers.legacy.Adam(
                learning_rate=hp.Float('learning_rate', 1e-5, 1e-3, sampling='LOG'),
                beta_1=hp.Float('beta_1', 0.90, 1, sampling='LOG'),
                beta_2=hp.Float('beta_2', 0.99, 1, sampling='LOG'),
                epsilon=hp.Float('epsilon', 1e-9, 1e-6, sampling='LOG')
            ),
            loss = loss,
            metrics=[BinaryAccuracy(), batch_iou]
        )

        return model

In [ ]:
hypermodel = CustomHyperModel()

In [ ]:
from keras_tuner.tuners import GridSearch, RandomSearch
from keras_tuner import Objective


tuner = RandomSearch(
    hypermodel,
    objective=Objective('val_loss', direction='min'),
    max_trials=10,  # количество комбинаций для проверки
    directory='FINAL_DIR',
    project_name='my_project',
    overwrite=False,
    seed=123
)

batch_size = 32
test_size = len(x_test) // batch_size * batch_size
train_size = len(x_train) // batch_size * batch_size
tuner.search(x_train[:train_size], y_train[:train_size], epochs=10, validation_data=(x_test[:test_size], y_test[:test_size]))

In [ ]:
import numpy as np

def batch_generator(x, y, batch_size):
    num_samples = x.shape[0]
    while True:
        # Перемешиваем индексы перед каждой эпохой
        indices = np.arange(num_samples)
        np.random.shuffle(indices)
        
        for start_idx in range(0, num_samples, batch_size):
            end_idx = min(start_idx + batch_size, num_samples)
            batch_indices = indices[start_idx:end_idx]
            x_batch = x[batch_indices]
            y_batch = y[batch_indices]
            yield x_batch, y_batch

In [ ]:
train_gen = batch_generator(x_train, y_train, batch_size)
validation_gen = batch_generator(x_test, y_test, batch_size)

In [ ]:
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hp)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

batch_size = 32

early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=1e-6, verbose=1)
model_checkpoint = ModelCheckpoint(filepath='model_{epoch:02d}_FINAL_01viewWDS.h5', period=5, save_best_only=True, save_weights_only=False, verbose=1)

    
history = best_model.fit_generator(
    train_gen,
    steps_per_epoch=len(x_train) // batch_size,
    epochs=200,
    validation_data=validation_gen,
    validation_steps=len(x_test) // batch_size,
    callbacks=[early_stopping, reduce_lr, model_check